# Modélisation #4 TabNet + HyperOpt

In [1]:
# Importation des packages

# Base
import importlib
import functions
importlib.reload(functions)
from functions import *

# Data Management
import pandas as pd
import numpy as np

# Modélisation
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

# Optimisation
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Importation des données

train_data = pd.read_csv('../data/Train_Data/6_train_data.csv', index_col=0)
train_scores = pd.read_csv('../data/Y_train_1rknArQ.csv', index_col=0)

sub_data = pd.read_csv('../data/Sub_Data/6_sub_data.csv', index_col=0)

train_data.head(5)

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,HOME_TEAM_YELLOWCARDS_season_sum,HOME_TEAM_REDCARDS_season_sum,HOME_TEAM_OFFSIDES_season_sum,HOME_TEAM_ATTACKS_season_sum,HOME_TEAM_PENALTIES_season_sum,HOME_TEAM_SUBSTITUTIONS_season_sum,HOME_TEAM_BALL_SAFE_season_sum,HOME_TEAM_DANGEROUS_ATTACKS_season_sum,HOME_TEAM_INJURIES_season_sum,HOME_TEAM_GOALS_season_sum,HOME_TEAM_GAME_WON_season_sum,HOME_TEAM_GAME_DRAW_season_sum,HOME_TEAM_GAME_LOST_season_sum,HOME_TEAM_SHOTS_TOTAL_season_average,HOME_TEAM_SHOTS_INSIDEBOX_season_average,HOME_TEAM_SHOTS_OFF_TARGET_season_average,HOME_TEAM_SHOTS_ON_TARGET_season_average,HOME_TEAM_SHOTS_OUTSIDEBOX_season_average,HOME_TEAM_PASSES_season_average,HOME_TEAM_SUCCESSFUL_PASSES_season_average,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average,HOME_TEAM_SAVES_season_average,HOME_TEAM_CORNERS_season_average,HOME_TEAM_BALL_POSSESSION_season_average,HOME_TEAM_FOULS_season_average,HOME_TEAM_YELLOWCARDS_season_average,HOME_TEAM_REDCARDS_season_average,HOME_TEAM_OFFSIDES_season_average,HOME_TEAM_ATTACKS_season_average,HOME_TEAM_PENALTIES_season_average,HOME_TEAM_SUBSTITUTIONS_season_average,HOME_TEAM_BALL_SAFE_season_average,HOME_TEAM_DANGEROUS_ATTACKS_season_average,HOME_TEAM_INJURIES_season_average,HOME_TEAM_GOALS_season_average,HOME_TEAM_GAME_WON_season_average,HOME_TEAM_GAME_DRAW_season_average,HOME_TEAM_GAME_LOST_season_average,HOME_TEAM_SHOTS_TOTAL_season_std,HOME_TEAM_SHOTS_INSIDEBOX_season_std,HOME_TEAM_SHOTS_OFF_TARGET_season_std,HOME_TEAM_SHOTS_ON_TARGET_season_std,HOME_TEAM_SHOTS_OUTSIDEBOX_season_std,HOME_TEAM_PASSES_season_std,HOME_TEAM_SUCCESSFUL_PASSES_season_std,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std,HOME_TEAM_SAVES_season_std,HOME_TEAM_CORNERS_season_std,HOME_TEAM_BALL_POSSESSION_season_std,HOME_TEAM_FOULS_season_std,HOME_TEAM_YELLOWCARDS_season_std,HOME_TEAM_REDCARDS_season_std,HOME_TEAM_OFFSIDES_season_std,HOME_TEAM_ATTACKS_season_std,HOME_TEAM_PENALTIES_season_std,HOME_TEAM_SUBSTITUTIONS_season_std,HOME_TEAM_BALL_SAFE_season_std,HOME_TEAM_DANGEROUS_ATTACKS_season_std,HOME_TEAM_INJURIES_season_std,HOME_TEAM_GOALS_season_std,HOME_TEAM_SHOTS_TOTAL_5_last_match_sum,HOME_TEAM_SHOTS_INSIDEBOX_5_last_match_sum,HOME_TEAM_SHOTS_OFF_TARGET_5_last_match_sum,HOME_TEAM_SHOTS_ON_TARGET_5_last_match_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum,HOME_TEAM_PASSES_5_last_match_sum,HOME_TEAM_SUCCESSFUL_PASSES_5_last_match_sum,HOME_TEAM_SAVES_5_last_match_sum,HOME_TEAM_CORNERS_5_last_match_sum,HOME_TEAM_FOULS_5_last_match_sum,HOME_TEAM_YELLOWCARDS_5_last_match_sum,HOME_TEAM_REDCARDS_5_last_match_sum,HOME_TEAM_OFFSIDES_5_last_match_sum,HOME_TEAM_ATTACKS_5_last_match_sum,HOME_TEAM_PENALTIES_5_last_match_sum,HOME_TEAM_SUBSTITUTIONS_5_last_match_sum,HOME_TEAM_BALL_SAFE_5_last_match_sum,HOME_TEAM_DANGEROUS_ATTACKS_5_last_match_sum,HOME_TEAM_INJURIES_5_last_match_sum,HOME_TEAM_GOALS_5_last_match_sum,HOME_TEAM_GAME_WON_5_last_match_sum,HOME_TEAM_GAME_DRAW_5_last_match_sum,HOME_TEAM_GAME_LOST_5_last_match_sum,HOME_TEAM_SHOTS_TOTAL_5_last_match_average,HOME_TEAM_SHOTS_INSIDEBOX_5_last_match_average,HOME_TEAM_SHOTS_OFF_TARGET_5_last_match_average,HOME_TEAM_SHOTS_ON_TARGET_5_last_match_average,HOME_TEAM_SHOTS_OUTSIDEBOX_5_last_match_average,HOME_TEAM_PASSES_5_last_match_average,HOME_TEAM_SUCCESSFUL_PASSES_5_last_match_average,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average,HOME_TEAM_SAVES_5_last_match_average,HOME_TEAM_CORNERS_5_last_match_average,HOME_TEAM_BALL_POSSESSION_5_last_match_average,HOME_TEAM_FOULS_5_last_match_average,HOME_TEAM_YELLOWCARDS_5_last_match_average,HOME_TEAM_REDCARDS_5_last_match_average,HOME_TEAM_OFFSIDES_5_last_match_average,HOME_TEAM_ATTACKS_5_last_match_average,HOME_TEAM_PENALTI

In [3]:
# Je préfère être sûr que les indexs concordent. 

train_scores = train_scores.loc[train_data.index]

train_scores.head(5)

,HOME_WINS,DRAW,AWAY_WINS
ID,,,
0,0,0,1
1,0,1,0
2,0,0,1
3,1,0,0
4,0,1,0


In [4]:
# Préparation des données pour la prédiction. 

train_scores_1c = train_scores[['HOME_WINS', 'DRAW', 'AWAY_WINS']].idxmax(axis=1)
label_mapping = {'HOME_WINS': 0, 'DRAW': 1, 'AWAY_WINS': 2}
train_scores_1c = train_scores_1c.replace(label_mapping)

train_scores_1c.head(5)

C:\Users\Hugo\AppData\Local\Temp\ipykernel_8144\2909335398.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_scores_1c = train_scores_1c.replace(label_mapping)


ID
0    2
1    1
2    2
3    0
4    1
dtype: int64

In [5]:
x_train, x_test, y_train, y_test = train_test_split(train_data, train_scores_1c, train_size=0.9, random_state=42)

Optimisation

In [6]:
# Définir l'espace de recherche des hyperparamètres

space = {
    'n_d': hp.quniform('n_d', 8, 64, 1),
    'n_a': hp.quniform('n_a', 8, 64, 1),
    'n_steps': hp.quniform('n_steps', 3, 10, 1),
    'gamma': hp.uniform('gamma', 1.0, 2.0),
    'lambda_sparse': hp.uniform('lambda_sparse', 0.00001, 0.01),
    'learning_rate': hp.uniform('learning_rate', 0.02, 0.2),
    'n_independent': hp.quniform('n_independent', 1, 5, 1),
    'n_shared': hp.quniform('n_shared', 1, 5, 1),
    'clip_value': hp.uniform('clip_value', 0, 2.0),
}

In [7]:
# Fonction d'optimisation

def objective(params):
    # Conversion des hyperparamètres avec les bonnes valeurs entières
    params['n_d'] = int(params['n_d'])
    params['n_a'] = int(params['n_a'])
    params['n_steps'] = int(params['n_steps'])
    params['n_independent'] = int(params['n_independent'])
    params['n_shared'] = int(params['n_shared'])

    # Initialiser le modèle TabNet avec les hyperparamètres
    model = TabNetClassifier(
        n_d=params['n_d'],
        n_a=params['n_a'],
        n_steps=params['n_steps'],
        gamma=params['gamma'],
        lambda_sparse=params['lambda_sparse'],
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=params['learning_rate']),
        n_independent=params['n_independent'],
        n_shared=params['n_shared'],
        clip_value=params['clip_value'],
        verbose=0
    )

    # Entraînement
        # Entraînement avec des hyperparamètres optimisés
    model.fit(
        x_train.values, y_train.values,
        max_epochs=500,  # Augmenter si nécessaire
        patience=30,     # Si aucun progrès pendant 30 epochs, arrêt
        batch_size=512,  # Taille de batch ajustée pour mémoire GPU
        virtual_batch_size=128,  # Taille virtuelle pour la gestion mémoire
        num_workers=2,  # Pour améliorer les performances d'entrée/sortie
    )

    # Prédiction
    preds = model.predict(x_test.values)
    acc = accuracy_score(y_test.values, preds)

    # Retourner une valeur à minimiser (on retourne -accuracy)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

  0%|          | 0/200 [00:00<?, ?trial/s, best loss=?]

c:\Users\Hugo\Documents\QRT_DATA_CHALLENGE\env_qrt\Lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



  0%|          | 1/200 [2:09:44<430:19:26, 7784.76s/trial, best loss: -0.4370430544272949]

c:\Users\Hugo\Documents\QRT_DATA_CHALLENGE\env_qrt\Lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



  0%|          | 1/200 [2:22:07<471:22:34, 8527.41s/trial, best loss: -0.4370430544272949]


KeyboardInterrupt: 

In [8]:
# Adaptation des paramètres trouvés par HyperOpt. 
best_params = {
    'n_d': int(best['n_d']),
    'n_a': int(best['n_a']),
    'n_steps': int(best['n_steps']),
    'gamma': best['gamma'],
    'lambda_sparse': best['lambda_sparse'],
    'n_independent': int(best['n_independent']),
    'n_shared': int(best['n_shared']),
    'clip_value': best['clip_value'],
    'learning_rate': best['learning_rate']
}

In [9]:
# Reformation du modèle

final_model = TabNetClassifier(
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    lambda_sparse=best_params['lambda_sparse'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_params['learning_rate']),
    n_independent=best_params['n_independent'],
    n_shared=best_params['n_shared'],
    clip_value=best_params['clip_value'],
    verbose=0
)

final_model.fit(x_train.values, y_train.values, max_epochs=200, patience=20, batch_size=1024, virtual_batch_size=128)


y_pred = final_model.predict(x_test.values)

/Users/hugopuybareau/Documents/GitHub/QRT_DATA_CHALLENGE/env_qrt/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


In [10]:
final_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy finale : {final_accuracy}")

Accuracy finale : 0.4900446972775295


# Prédictions

In [11]:
y_sub = final_model.predict(sub_data.values)

In [12]:
# Reconstruction du DataFrame y_sub avec les colonnes 'HOME_WINS', 'DRAW', 'AWAY_WINS'
y_sub_df = pd.DataFrame(y_sub, columns=['PRED'])

# Transformation en one-hot encoding pour obtenir les 3 colonnes
y_sub_df['HOME_WINS'] = (y_sub_df['PRED'] == 0).astype(int)
y_sub_df['DRAW'] = (y_sub_df['PRED'] == 1).astype(int)
y_sub_df['AWAY_WINS'] = (y_sub_df['PRED'] == 2).astype(int)

# Ajout de la colonne 'ID' depuis sub_data
y_sub_df['ID'] = sub_data.index

# Suppression de la colonne 'PRED'
y_sub_df.drop('PRED', axis=1, inplace=True)

# Réorganiser les colonnes pour avoir 'ID' en premier
y_sub_df = y_sub_df[['ID', 'HOME_WINS', 'DRAW', 'AWAY_WINS']]

# Afficher les premières lignes du résultat final
y_sub_df = y_sub_df.set_index('ID')
y_sub_df.head()

,HOME_WINS,DRAW,AWAY_WINS
ID,,,
12303.0,1,0,0
12304.0,0,0,1
12305.0,1,0,0
12306.0,1,0,0
12307.0,1,0,0


In [13]:
y_sub_df.to_csv('../data/Sub_Data/14_sub_scores.csv')